<a href="https://colab.research.google.com/github/GaurangRawat/Shipclues/blob/main/Shipclues_Chatbot_p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install langchain groq pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 4.3 MB/s eta 0:00:00


In [20]:
import os
from google.colab import userdata


os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")


In [4]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


In [21]:
!pip install langchain_groq

In [29]:
import pandas as pd
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_groq import ChatGroq  # Groq LLM wrapper

# Load CSV
df = pd.read_csv("amazon_delivery.csv")

# Tool function to look up order details
def lookup_order(order_id: str) -> str:
    order_id_clean = order_id.strip().lower()
    result = df[df["Order_ID"] == order_id_clean]

    if result.empty:
        return f"❌ No order found with ID: {order_id}"

    # Format selected details
    order = result.iloc[0]
    return (f"📝 Order ID: {order['Order_ID']}\n"
            f"📅 Date: {order['Order_Date']}, ⏰ Time: {order['Order_Time']}\n"
            f"🚚 Delivery Time: {order['Delivery_Time']} mins\n"
            f"🌤️ Weather: {order['Weather']}, 🚦 Traffic: {order['Traffic']}\n"
            f"🧑 Agent Age: {order['Agent_Age']}, Rating: {order['Agent_Rating']}")

# Define tool for LangChain
tools = [
    Tool(
        name="Order Lookup",
        func=lookup_order,
        description="Use this tool to look up delivery details for a given order ID. Input should be the order ID."
    )
]

# Load open-source LLM from Groq (fast & free)
llm = ChatGroq(
    model_name="llama3-8b-8192",  # or use "mixtral-8x7b", "gemma-7b-it"
    temperature=0.3
)

# Create LangChain agent with tool
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Example prompt
user_prompt = "What was the delivery time and weather condition for order: zguw716275638  ?"
response = agent.run(user_prompt)
print("\n🧾 Answer:", response)




> Entering new AgentExecutor chain...
Thought: I need to look up the delivery details for the given order ID to find the delivery time and weather condition.

Action: Order Lookup
Action Input: zguw716275638

Observation: 📝 Order ID: zguw716275638
📅 Date: 2022-03-26, ⏰ Time: 13:30:00
🚚 Delivery Time: 150 mins
🌤️ Weather: Cloudy, 🚦 Traffic: High 
🧑 Agent Age: 32, Rating: 4.6
Thought:Here's the answer:

Thought: I need to look up the delivery details for the given order ID to find the delivery time and weather condition.

Action: Order Lookup
Action Input: zguw716275638

Observation: 📝 Order ID: zguw716275638
📅 Date: 2022-03-26, ⏰ Time: 13:30:00
🚚 Delivery Time: 150 mins
🌤️ Weather: Cloudy, 🚦 Traffic: High 
🧑 Agent Age: 32, Rating: 4.6
Thought:Thought: I now know the final answer.

Final Answer: The delivery time for order zguw716275638 was 150 minutes, and the weather condition was Cloudy with High traffic.

> Finished chain.

🧾 Answer: The delivery time for order zguw716275638 was 15

In [30]:
def is_delivery_related(query: str) -> bool:
    delivery_keywords = ["order", "delivery", "vehicle", "agent", "traffic", "weather", "category"]
    return any(keyword in query.lower() for keyword in delivery_keywords)

# Safe interaction
user_query = input("Ask a question: ")
if is_delivery_related(user_query):
    print(agent.run(user_query))
else:
    print("❌ I can only help with delivery-related questions.")


Ask a question: details about order : ialx566343618


> Entering new AgentExecutor chain...
Thought: I need to look up the delivery details for the given order ID "ialx566343618".

Action: Order Lookup
Action Input: ialx566343618

Observation: 📝 Order ID: ialx566343618
📅 Date: 2022-03-19, ⏰ Time: 11:30:00
🚚 Delivery Time: 120 mins
🌤️ Weather: Sunny, 🚦 Traffic: High 
🧑 Agent Age: 37, Rating: 4.9
Thought:Thought: I now know the delivery details for the given order ID "ialx566343618".

Final Answer: The delivery details for order "ialx566343618" are: Order ID: ialx566343618, Date: 2022-03-19, Time: 11:30:00, Delivery Time: 120 mins, Weather: Sunny, Traffic: High, Agent Age: 37, Rating: 4.9.

> Finished chain.
The delivery details for order "ialx566343618" are: Order ID: ialx566343618, Date: 2022-03-19, Time: 11:30:00, Delivery Time: 120 mins, Weather: Sunny, Traffic: High, Agent Age: 37, Rating: 4.9.
